In [1]:
import pandas as pd

In [20]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [21]:
users = df[['person']].drop_duplicates().copy()
users.set_index('person', inplace=True)

In [24]:
df_ult_quin = df.loc[df['timestamp'] > pd.to_datetime('2018-05-15'), :]

In [25]:
def calcular_freq(evento):
    df_colores = df_ult_quin.loc[df['event'] == evento, :].groupby('person')['color'].value_counts()
    df_col_mas_evento = df_colores.rename(columns={'color':'max'}).reset_index().groupby('person').first()[['color']].reset_index()
    df_col_mas_evento.rename(columns={'color':'color_'+evento}, inplace=True)
    dict_colores = df_col_mas_evento['color_'+evento].value_counts().to_dict()
    df_col_mas_evento.set_index('person', inplace=True)
    df_col_mas_evento = df_col_mas_evento.applymap(lambda s: dict_colores.get(s) if s in dict_colores else s)
    return df_col_mas_evento

In [26]:
df_checkout = calcular_freq('checkout')
df_conversion = calcular_freq('conversion')
df_viewed_prod = calcular_freq('viewed product')

In [27]:
df_final = users.join(df_checkout).fillna(0)
df_final = df_final.join(df_conversion).fillna(0)
df_final = df_final.join(df_viewed_prod).fillna(0)

In [28]:
df_final.shape

(38829, 3)

In [29]:
df_final.to_csv('../data/feature_color_ult15dias.csv')

In [30]:
users.shape

(38829, 0)